In [ ]:
#connect to the googee drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install rasterio geopandas

In [ ]:
#import
import os
import rasterio 
#import rasterio.warp             # Reproject raster samples
from rasterio import windows
#import geopandas as gps
#import PIL.Image
#import PIL.ImageDraw
import fiona 
import imageio
import gdal

import sys
sys.path.insert(0,"/content/drive/MyDrive/Colab/Wildebeest_UNet")
import unet_wildebeest
from unet_wildebeest import *

from shapely.geometry import Point, MultiPoint
from shapely.geometry import mapping, shape
import numpy as np               # numerical array manipulation
#from tqdm import tqdm
import cv2
import random
from rasterio.windows import Window
from skimage import measure
from sklearn.cluster import KMeans

#Define path for test dataset, classify dataset

In [ ]:
Data_folder = "/content/drive/My Drive/Colab/Wildebeest_UNet"
IMAGE_PATH = os.path.join(Data_folder, "data/test2/images")
LABEL_PATH = os.path.join(Data_folder, "data/test2/masks")

In [ ]:
CLASSIFY_PATH = "/content/drive/MyDrive/Colab/Wildebeest_UNet/data/classify/images"

In [ ]:
'''
image = imageio.imread("/content/drive/MyDrive/Colab/Wildebeest_UNet/accuracy_sample/true009.png")
print(np.shape(image))
mask = rasterio.open("/content/drive/MyDrive/Colab/Wildebeest_UNet/accuracy_sample/image009.tif")
predict = imageio.imread("/content/drive/MyDrive/Colab/Wildebeest_UNet/accuracy_sample/predict009.png")
'''

(336, 336)


#Testing dataset accuracy assessment

##Load test dataset

In [ ]:
#Read test images
# create tiles and read array (keep the array and crs at the same time)
PATCHSIZE=336
test_list=np.zeros(shape=(0, PATCHSIZE, PATCHSIZE, 4),
                            dtype=np.uint8)
meta_list=[]
for f in sorted(os.listdir(IMAGE_PATH)):
    fdir = os.path.join(IMAGE_PATH, f)
    _, ext = os.path.splitext(f)
    if ext.lower() == ".tif":
        #imgtype = f[-7:-4]
        #print(imgtype)
        image_data=gdal.Open(fdir)
        bands = [image_data.GetRasterBand(i+1).ReadAsArray() for i in range(image_data.RasterCount)]
        new_image = np.stack(bands, axis=2)
        new_image = np.expand_dims(new_image[0:PATCHSIZE,0:PATCHSIZE,:], 0)
        test_list = np.concatenate((test_list, new_image), axis=0)

        src=rasterio.open(fdir)
        # The size in pixels of your desired window
        xsize, ysize = PATCHSIZE, PATCHSIZE

        # Generate a random window location that doesn't go outside the image
        xmin, xmax = 0, src.width - xsize
        ymin, ymax = 0, src.height - ysize
        xoff, yoff = random.randint(xmin, xmax), random.randint(ymin, ymax)

        # Create a Window and calculate the transform from the source dataset    
        window = Window(xoff, yoff, xsize, ysize)
        transform = src.window_transform(window)

        # Create a new cropped raster to write to
        profile = src.profile
        profile.update({
           'height': xsize,
           'width': ysize,
           'transform': transform})
        meta_list = np.append(meta_list, src)

In [ ]:
print(np.shape(test_list))
print(np.shape(meta_list))
print(type(meta_list[1]))

(30, 336, 336, 4)
(30,)
<class 'rasterio.io.DatasetReader'>


In [ ]:
label_list=np.zeros(shape=(0, PATCHSIZE, PATCHSIZE, 1),
                            dtype=np.uint8)
for f in sorted(os.listdir(LABEL_PATH)):
    fdir = os.path.join(LABEL_PATH, f)
    _, ext = os.path.splitext(f)
    if ext.lower() == ".tif":
        #imgtype = f[-7:-4]
        #print(imgtype)
        image_data=gdal.Open(fdir)
        bands = [image_data.GetRasterBand(i+1).ReadAsArray() for i in range(image_data.RasterCount)]
        new_image = np.stack(bands, axis=2)
        new_image = np.expand_dims(new_image[0:PATCHSIZE,0:PATCHSIZE,:], 0)
        label_list = np.concatenate((label_list, new_image), axis=0)

In [ ]:
Xtest = np.float32(test_list)
Xtest_contrast = seq_contrast(images=Xtest)
Ytest = np.float32(label_list)

##Predict on test dataset

In [ ]:
model = unet(pretrained_weights=None, input_size=(PATCHSIZE,PATCHSIZE,4))
#model = load_model("/content/drive/MyDrive/Colab/Wildebeest_UNet/model/unet_AdaDelta_tversky.hdf5", custom_objects={'tversky': tversky, 'dice_coef': dice_coef, 'dice_loss':dice_loss, 'accuracy':accuracy , 'precision':precision_m, 'recall':recall_m, 'f1-score':f1_m}, compile=False)
#model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_coef, dice_loss,accuracy, recall_m, precision_m, f1_m])
#model.load_weights("/content/drive/MyDrive/Colab/Wildebeest_UNet/model/unet_AdaDelta_tversky_fullTrainingEpoch66.hdf5")
#model.load_weights("/content/drive/MyDrive/Colab/Wildebeest_UNet/tmp/checkpoint/weights.103-0.70.hdf5")
model.load_weights("/content/drive/MyDrive/Colab/Wildebeest_UNet/tmp/checkpoint_subTrain30Test_adam0.0001_reducelr/weights.133-0.26.hdf5")
Ypredict = model.predict(Xtest)
Ypredict[Ypredict>0.5]=1.0
Ypredict[Ypredict<=0.5]=0.0

In [ ]:
Ypredict_contrast = model.predict(Xtest_contrast)
Ypredict_contrast[Ypredict_contrast>0.5]=1.0
Ypredict_contrast[Ypredict_contrast<=0.5]=0.0

In [ ]:
def print_pixel_accuracy(predictions, Ytest):
  print("TP: ", K.sum(true_positives(Ytest, predictions)))
  print("FP: ", K.sum(false_positives(Ytest, predictions)))
  print("TN: ", K.sum(true_negatives(Ytest, predictions)))
  print("FN: ", K.sum(false_negatives(Ytest, predictions)))
  print("precision_m: ",K.sum(precision_m(Ytest, predictions)))
  print("recall_m: " ,K.sum(recall_m(Ytest, predictions)))
  print("f1_m: " ,K.sum(f1_m(Ytest, predictions)))

In [ ]:
print_pixel_accuracy(Ypredict, Ytest)
print_pixel_accuracy(Ypredict_contrast, Ytest)

TP:  tf.Tensor(18792.0, shape=(), dtype=float32)
FP:  tf.Tensor(5986.0, shape=(), dtype=float32)
TN:  tf.Tensor(3353601.0, shape=(), dtype=float32)
FN:  tf.Tensor(8501.0, shape=(), dtype=float32)
precision_m:  tf.Tensor(0.75841475, shape=(), dtype=float32)
recall_m:  tf.Tensor(0.6885282, shape=(), dtype=float32)
f1_m:  tf.Tensor(0.72178364, shape=(), dtype=float32)
TP:  tf.Tensor(13063.0, shape=(), dtype=float32)
FP:  tf.Tensor(2625.0, shape=(), dtype=float32)
TN:  tf.Tensor(3356962.0, shape=(), dtype=float32)
FN:  tf.Tensor(14230.0, shape=(), dtype=float32)
precision_m:  tf.Tensor(0.8326747, shape=(), dtype=float32)
recall_m:  tf.Tensor(0.4786209, shape=(), dtype=float32)
f1_m:  tf.Tensor(0.60784996, shape=(), dtype=float32)


In [ ]:
print(np.shape(Ypredict))

(12, 336, 336, 1)


In [ ]:
scores = model.evaluate(Xtest, Ytest, verbose=0)
print(scores)
scores2 = model.evaluate(Xtest_contrast, Ytest, verbose=0)
print(scores2)

[0.2837119996547699, 0.011973235756158829, 0.988025963306427, 0.9895508289337158, 0.6013686060905457, 0.7918285131454468, 0.6835799217224121]
[0.6123940944671631, 0.003776039695367217, 0.9962228536605835, 0.9840826988220215, 0.17005544900894165, 0.9036572575569153, 0.28624385595321655]


##Point accuracy evaluation

In [ ]:
#Now you have Ytest (true labels array), Ypredict (prediction array), meta_list (raster with crs). All are ordered.
# Next: Ytest -> points; Ypredict -> points
'''
def transformCentroidToXYpoints(centroid, transform = None):
    points = []
    for cnt in centroid:
        cols, rows = zip(*cnt)
        x,y = rasterio.transform.xy(transform, rows, cols)
        point = Point(float(x[0]), float(y[0]))
        #tl = [list(i) for i in zip(x, y)]
        points.append(point)
    return (points)
'''

def ImageToPoints(image, mask):
  # find the contours of the image segments
  image = image.astype(np.uint8)
  #contours, hierarchy = cv2.findContours(image, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

  # calculate the centroid of each contour, and save them in wildebeest list
  #wildebeest = np.zeros(shape=(0,1,2),dtype=np.uint8)
  transform = mask.meta['transform']
  wildebeest = []
  labels = measure.label(image)
  regions = measure.regionprops(labels)
  for region in regions:
    #print(region.centroid)
    num = np.int(np.ceil(region.area/9))
    if num == 1:
      centroid = list(np.round(region.centroid))
      wildebeest.append(centroid)
    else:
      clusters = KMeans(num).fit(region.coords)
      centroids = np.round(clusters.cluster_centers_)
      for centroid in centroids:
        centroid = list(centroid)
        wildebeest.append(centroid)
  points = []
  for point in wildebeest:
    #rows, cols = zip(*centroid)
    x,y = rasterio.transform.xy(transform, point[0], point[1])
    point = Point(x, y)
    points.append(point)

    #centroid = np.expand_dims(centroid, axis=0)
    #wildebeest = np.concatenate((wildebeest, centroid), axis=0)
    #print(wildebeest)

    #wildebeest = transformCentroidToXYpoints(wildebeest, meta['transform'])
  return points

In [ ]:
def nearest_neighbor_within(others, point, max_distance):
    """Find nearest point among others up to a maximum distance.

    Args:
        others: a list of Points or a MultiPoint
        point: a Point
        max_distance: maximum distance to search for the nearest neighbor

    Returns:
        A shapely Point if one is within max_distance, None otherwise
    """
    search_region = point.buffer(max_distance)
    interesting_points = search_region.intersection(MultiPoint(others))

    if not interesting_points:
        closest_point = None
    elif isinstance(interesting_points, Point):
        closest_point = interesting_points
    else:
        distances = [point.distance(ip) for ip in interesting_points
                     if point.distance(ip) > 0]
        closest_point = interesting_points[distances.index(min(distances))]

    return closest_point 


import tensorflow.keras.backend as K
def evaluation(true_points, predict_points, threshold = 0.8):
  True_Positives = []
  False_Positives = []
  False_Negatives = []
  positives = predict_points.copy()
  for true_point in true_points:
    true_positive = nearest_neighbor_within(positives, true_point, threshold)
    if true_positive == None:
     False_Negatives.append(true_point)
    else:
     True_Positives.append(true_positive)
     positives.remove(true_positive)
  False_Positives = positives
  TP = len(True_Positives)
  FP = len(False_Positives)
  FN = len(False_Negatives)
  if TP == 0 and FP == 0:
    Precision = 1
  else:
    Precision = float(TP/(TP+FP))
  if TP == 0 and FN == 0:
    Recall = 1
  else:
    Recall = float(TP/(TP+FN))
  F1 = 2*(Precision*Recall)/(Precision+Recall+K.epsilon())
  accuracy = {
      "TP": TP,
      "FP": FP,
      "FN": FN,
      "Precision":Precision,
      "Recall":Recall,
      "F1":F1
  }
  return accuracy


In [ ]:
Total_TP = 0
Total_FP = 0
Total_FN = 0
for i in range(Xtest.shape[0]):
  print('Test image: ', i)
  true_pts = ImageToPoints(Ytest[i],meta_list[i])
  predict_pts = ImageToPoints(Ypredict[i],meta_list[i])
  accuracy = evaluation(true_pts, predict_pts, threshold=0.7)
  print(accuracy)
  Total_TP += accuracy['TP']
  Total_FP += accuracy['FP']
  Total_FN += accuracy['FN']
Total_precision = Total_TP/(Total_TP+Total_FP)
Total_recall = Total_TP/(Total_TP+Total_FN)
Total_f1 = 2*(Total_precision*Total_recall)/(Total_precision+Total_recall+K.epsilon())
print(Total_precision, Total_recall, Total_f1)

Test image:  0
{'TP': 0, 'FP': 0, 'FN': 0, 'Precision': 1, 'Recall': 1, 'F1': 0.9999999500000026}
Test image:  1
{'TP': 0, 'FP': 6, 'FN': 0, 'Precision': 0.0, 'Recall': 1, 'F1': 0.0}
Test image:  2
{'TP': 85, 'FP': 16, 'FN': 37, 'Precision': 0.8415841584158416, 'Recall': 0.6967213114754098, 'F1': 0.7623317890084288}
Test image:  3
{'TP': 395, 'FP': 255, 'FN': 24, 'Precision': 0.6076923076923076, 'Recall': 0.9427207637231504, 'F1': 0.7390083714179985}
Test image:  4
{'TP': 63, 'FP': 2, 'FN': 59, 'Precision': 0.9692307692307692, 'Recall': 0.5163934426229508, 'F1': 0.6737967460893965}
Test image:  5
{'TP': 403, 'FP': 64, 'FN': 78, 'Precision': 0.8629550321199143, 'Recall': 0.8378378378378378, 'F1': 0.8502109204750424}
Test image:  6
{'TP': 54, 'FP': 11, 'FN': 72, 'Precision': 0.8307692307692308, 'Recall': 0.42857142857142855, 'F1': 0.5654449812779291}
Test image:  7
{'TP': 19, 'FP': 11, 'FN': 12, 'Precision': 0.6333333333333333, 'Recall': 0.6129032258064516, 'F1': 0.6229507696855725}
Test

In [ ]:
Total_TP = 0
Total_FP = 0
Total_FN = 0
for i in range(Xtest.shape[0]):
  print('Test image: ', i)
  true_pts = ImageToPoints(Ytest[i],meta_list[i])
  predict_pts = ImageToPoints(Ypredict_contrast[i],meta_list[i])
  accuracy = evaluation(true_pts, predict_pts, threshold=0.7)
  print(accuracy)
  Total_TP += accuracy['TP']
  Total_FP += accuracy['FP']
  Total_FN += accuracy['FN']
Total_precision = Total_TP/(Total_TP+Total_FP)
Total_recall = Total_TP/(Total_TP+Total_FN)
Total_f1 = 2*(Total_precision*Total_recall)/(Total_precision+Total_recall+K.epsilon())
print(Total_precision, Total_recall, Total_f1)

Test image:  0
{'TP': 0, 'FP': 0, 'FN': 0, 'Precision': 1, 'Recall': 1, 'F1': 0.9999999500000026}
Test image:  1
{'TP': 0, 'FP': 4, 'FN': 0, 'Precision': 0.0, 'Recall': 1, 'F1': 0.0}
Test image:  2
{'TP': 55, 'FP': 13, 'FN': 67, 'Precision': 0.8088235294117647, 'Recall': 0.45081967213114754, 'F1': 0.5789473224598375}
Test image:  3
{'TP': 268, 'FP': 111, 'FN': 151, 'Precision': 0.7071240105540897, 'Recall': 0.639618138424821, 'F1': 0.6716791481206185}
Test image:  4
{'TP': 33, 'FP': 0, 'FN': 89, 'Precision': 1.0, 'Recall': 0.27049180327868855, 'F1': 0.42580641809781744}
Test image:  5
{'TP': 226, 'FP': 13, 'FN': 255, 'Precision': 0.9456066945606695, 'Recall': 0.4698544698544699, 'F1': 0.6277777334263148}
Test image:  6
{'TP': 64, 'FP': 18, 'FN': 62, 'Precision': 0.7804878048780488, 'Recall': 0.5079365079365079, 'F1': 0.6153845676220451}
Test image:  7
{'TP': 17, 'FP': 9, 'FN': 14, 'Precision': 0.6538461538461539, 'Recall': 0.5483870967741935, 'F1': 0.5964911784549134}
Test image:  8
{'

##Generate point shapefile

In [ ]:
# use the points generated from ImageToPoints(), generate the point shapefile
schema = {
    'geometry': 'Point',
    'properties': {'id': 'str'},
    }
def createShapefileObject(points, meta, wfile):
    with fiona.open(wfile, 'w', crs=meta.get('crs').to_dict(), driver='ESRI Shapefile', schema=schema) as sink:
        for idx, point in enumerate(points):
            sink.write({
                'geometry': mapping(point),
                'properties': {'id': str(idx)},
                })
            #print(mapping(point))

In [ ]:
points009 = ImageToPoints(image, mask)
print(len(points009))

181


In [ ]:
createShapefileObject(predict009, mask.meta, os.path.join(Data_folder, 'predict009.shp'))

In [ ]:
createShapefileObject(points009, mask.meta, os.path.join(Data_folder, 'true009.shp'))

In [ ]:
true_pts = ImageToPoints(Ytest[13],meta_list[13])
predict_pts = ImageToPoints(Ypredict[13],meta_list[13])

In [ ]:
createShapefileObject(true_pts, meta_list[13].meta, os.path.join(Data_folder, 'data/points/true014.shp'))
createShapefileObject(predict_pts, meta_list[13].meta, os.path.join(Data_folder, 'data/points/predict014.shp'))

In [ ]:
createShapefileObject(TRUE_points[2], meta_list[2].meta, os.path.join(Data_folder, 'true010.shp'))
createShapefileObject(PREDICT_points[2], meta_list[2].meta, os.path.join(Data_folder, 'predict010.shp'))

#Classify satellite images
##Load classify dataset

In [ ]:
classify_list=np.zeros(shape=(0, PATCHSIZE, PATCHSIZE, 4),
                            dtype=np.uint8)

for f in sorted(os.listdir(CLASSIFY_PATH)):
    fdir = os.path.join(CLASSIFY_PATH, f)
    _, ext = os.path.splitext(f)
    if ext.lower() == ".tif":
        #imgtype = f[-7:-4]
        #print(imgtype)
        image_data=gdal.Open(fdir)
        bands = [image_data.GetRasterBand(i+1).ReadAsArray() for i in range(image_data.RasterCount)]
        new_image = np.stack(bands, axis=2)
        new_image = np.expand_dims(new_image[0:PATCHSIZE,0:PATCHSIZE,:], 0)
        classify_list = np.concatenate((classify_list, new_image), axis=0)       

In [ ]:
print(np.shape(classify_list))

(120, 336, 336, 4)


In [ ]:
Xclassify = np.float32(classify_list)
Xclassify_contrast = seq_contrast(images=Xclassify)

In [ ]:
model = unet(pretrained_weights=None, input_size=(PATCHSIZE,PATCHSIZE,4))
#model = load_model("/content/drive/MyDrive/Colab/Wildebeest_UNet/model/unet_AdaDelta_tversky.hdf5", custom_objects={'tversky': tversky, 'dice_coef': dice_coef, 'dice_loss':dice_loss, 'accuracy':accuracy , 'precision':precision_m, 'recall':recall_m, 'f1-score':f1_m}, compile=False)
#model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_coef, dice_loss,accuracy, recall_m, precision_m, f1_m])
model.load_weights("/content/drive/MyDrive/Colab/Wildebeest_UNet/model/unet_AdaDelta_tversky_fullTrainingEpoch66.hdf5")
#model.load_weights("/content/drive/MyDrive/Colab/Wildebeest_UNet/tmp/checkpoint/weights.103-0.70.hdf5")
predict = model.predict(Xclassify)
predict[predict>0.5]=1.0
predict[predict<=0.5]=0.0

In [ ]:
print(np.shape(predict))

(120, 336, 336, 1)


In [ ]:
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
def visualize_data(data,data2):
    f, axarr = plt.subplots(1,2)
    data = data[:, :, 0:3]
    axarr[0].imshow(data)
    axarr[0].set_title("Satellite image")

    a=axarr[1]
    values = np.unique(data2.ravel())
    im = axarr[1].imshow(data2[:,:,0])
    a.set_title("Labeled image")
    colors = [im.cmap(im.norm(value)) for value in values] 
    #colors = [im.cmap(value) for value in values]
    data2 = ["Non-wildebeest", "wildebeest"]
    patches = [mpatches.Patch(color=colors[i], label=j) for i, j in zip(range(len(values)), data2)]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['shape', 'add', 'concatenate', 'multiply', 'minimum', 'average', 'maximum', 'dot', 'random', 'f', 'subtract']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
for i in range(0,10):
  visualize_data(classify_list[i].astype('uint8'),predict[i,:,:,:])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in range(10,20):
  visualize_data(classify_list[i].astype('uint8'),predict[i,:,:,:])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in range(20,30):
  visualize_data(classify_list[i].astype('uint8'),predict[i,:,:,:])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in range(30,40):
  visualize_data(classify_list[i].astype('uint8'),predict[i,:,:,:])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in range(40,50):
  visualize_data(classify_list[i].astype('uint8'),predict[i,:,:,:])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in range(50,60):
  visualize_data(classify_list[i].astype('uint8'),predict[i,:,:,:])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in range(60,70):
  visualize_data(classify_list[i].astype('uint8'),predict[i,:,:,:])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in range(70,80):
  visualize_data(classify_list[i].astype('uint8'),predict[i,:,:,:])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in range(70,80):
  visualize_data(classify_list[i].astype('uint8'),predict[i,:,:,:])